In [23]:
import numpy as np
from itertools import product, permutations, combinations

In [9]:
with open('day19.txt') as f:
    data = f.read()

In [10]:
scanners = [scanner.splitlines()[1:] for scanner in data.split('\n\n')]
scanners = [{tuple(map(int, line.split(','))) for line in scanner} for scanner in scanners]

In [11]:
def permutation_parity(arr):
    arr = np.asarray(arr)
    l = arr.size
    i, j = np.tril_indices(l, -1)
    return np.product(np.sign(arr[i] - arr[j]))

def generate_rotations2():
    for permut in permutations(range(3)):
        for signs in product([-1, 1], repeat=2):
            sign_parity = 1 if signs.count(-1) % 2 == 0 else -1
            last_sign = permutation_parity(permut) * sign_parity
            signs = signs + (last_sign,)
            yield np.eye(3, dtype=int)[:, permut] * signs

In [12]:
def generate_rotations(sc):
    for permut in permutations(range(3)):
        for sign in product([-1, 1], repeat=3):
            rot_mat = np.eye(3, dtype=int)[:, permut] * sign
            if np.linalg.det(rot_mat) == 1:
                yield {tuple(rot_mat @ beacon) for beacon in sc}

In [13]:
def sub(x, y):
    return tuple(a - b for a, b in zip(x, y))

In [16]:
def match(sc1, sc2):
    for sc2 in generate_rotations(sc2):
        for b1 in sc1:
            for b2 in sc2:
                offset = sub(b2, b1)
                b2_off = {sub(b, offset) for b in sc2}
                if len(b2_off & sc1) >= 12:
                    return b2_off, offset
    return None, None

In [17]:
beacons = scanners[0]
rest = scanners[1:]
offsets = [(0,0,0)]

while rest:
    matched, offset = match(beacons, rest[0])
    if matched:
        beacons |= matched
        rest.pop(0)
        offsets.append(offset)
    else:
        rest.append(rest.pop(0))

In [18]:
len(beacons)

365

In [36]:
max_ = 0
for c in combinations(range(len(offsets)), 2):
    max_ = max(max_, sum([np.abs(a - b) for a, b in zip(offsets[c[0]], offsets[c[1]])]))
max_

11060